In [1]:
import tensorflow as tf
import pandas as pd
from molgraph.models import GradientActivationMapping
from molgraph.chemistry import vis
from helpers import encode, encoder

def make_gradcam_heatmap(input_structure, model=None, return_prob=False):
    gam_model = GradientActivationMapping(
        model,
        [i.name for i in model.layers if "conv" in i.name], # all conv layers by default
        output_activation=None,
        discard_negative_values=True
    )
    graph = encode(input_structure)
    gam = gam_model(graph)
    if return_prob:
        return model.predict(graph)
    return vis.visualize_maps(molecule=input_structure, maps=gam[0])


model_file = "../saved_models/GCNII"
model = tf.keras.models.load_model(model_file)

2024-01-05 20:32:12.668342: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 20:32:12.669573: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 20:32:12.693239: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 20:32:12.693879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 20:32:13.161872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
from rdkit import Chem
df = pd.read_csv("../data/SMILES_test/test_data_all.csv")
df["mol"] = df.SMILES.apply(Chem.MolFromSmiles)
# .query("covalent == 0 and warhead_category == 'noncovalentdecoy'")
probs = model.predict(encoder(df.InChI.values))
df["prob"] = probs

20/20 [==============================] - 4s 6ms/step


,SMILES,InChI,warhead_category,covalent,mol,prob
0,[H]C#CN([H])C(=O)c1nc(N2C([H])([H])C([H])([H])...,InChI=1S/C25H26N8O3/c1-2-26-24(36)20-13-22(28-...,aldehyde,1,<rdkit.Chem.rdchem.Mol object at 0x7f07fbdf3df0>,0.995669
1,[H]C#CN([H])C(=O)c1nc(N2C([H])([H])C([H])([H])...,InChI=1S/C25H26N8O3/c1-2-26-24(36)20-13-22(28-...,aldehyde,1,<rdkit.Chem.rdchem.Mol object at 0x7f07fbdf3e60>,0.994756
2,[H]Oc1c([H])c(OC([H])([H])[H])c(-c2c([H])n([H]...,InChI=1S/C23H20N4O4/c1-27(2)23(30)15-4-13(8-24...,aldehyde,1,<rdkit.Chem.rdchem.Mol object at 0x7f07fbdf3ed0>,0.179605
3,[H]C(=O)c1c([H])c([H])c(S(=O)(=O)N2C([H])([H])...,InChI=1S/C13H17NO4S/c1-10-7-14(8-11(2)18-10)19...,aldehyde,1,<rdkit.Chem.rdchem.Mol object at 0x7f07fbdf3f40>,0.890543
4,[H]C(=O)c1nc2c(C(=O)N([H])c3nc([H])c(C(F)(F)F)...,"InChI=1S/C17H10F3N3O2/c18-17(19,20)11-5-7-14(2...",aldehyde,1,<rdkit.Chem.rdchem.Mol object at 0x7f07fa7a4040>,0.576873
...,...,...,...,...,...,...
605,[H]C([H])([H])C1=NC([H])([H])C([H])([H])C1([H]...,"InChI=1S/C5H9N/c1-5-3-2-4-6-5/h2-4H2,1H3",noncovalentdecoy,0,<rdkit.Chem.rdchem.Mol object at 0x7f07fa7b4970>,0.000118
606,[H]OC([H])([H])[C@@]1([H])O[C@]1(C([H])([H])[H...,InChI=1S/C10H18O2/c1-8(2)5-4-6-10(3)9(7-11)12-...,noncovalentdecoy,0,<rdkit.Chem.rdchem.Mol object at 0x7f07fa7b49e0>,0.141947
607,[H]OC([H])([H])/C([H])=C(\C([H])([H])[H])C([H]...,"InChI=1S/C10H18O2/c1-8(6-7-11)4-5-9-10(2,3)12-...",noncovalentdecoy,0,<rdkit.Chem.rdchem.Mol object at 0x7f07fa7b4a50>,0.038306
608,[H]OC([H])([H])C([H])([H])[C@]([H])(C([H])([H]...,"InChI=1S/C10H20O2/c1-8(6-7-11)4-5-9-10(2,3)12-...",noncovalentdecoy,0,<rdkit.Chem.rdchem.Mol object at 0x7f07fa7b4ac0>,0.018422


In [ ]:
df = df.sort_values(by="prob", ascending=False).reset_index(drop=True)
df

In [ ]:
from rdkit import Chem
make_gradcam_heatmap(Chem.MolToSmiles(Chem.MolFromInchi(df.InChI[14])), model=model)

In [ ]:
make_gradcam_heatmap(df.InChI[14], model=model)

In [ ]:
ibrutnib = "InChI=1S/C22H22N6O3/c1-4-19(29)27-8-7-15(12-27)28-22-20(21(23)24-13-25-22)18(26-28)6-5-14-9-16(30-2)11-17(10-14)31-3/h4,9-11,13,15H,1,7-8,12H2,2-3H3,(H2,23,24,25)/t15-/m0/s1"
ganfeborole = "InChI=1S/C10H13BClNO4/c12-6-1-2-7(16-4-3-14)10-9(6)8(5-13)17-11(10)15/h1-2,8,14-15H,3-5,13H2/t8-/m1/s1"
chembl8796 = "InChI=1S/C12H19N3O/c13-9-15-7-10(8-15)6-14-12(16)11-4-2-1-3-5-11/h10-11H,1-8H2,(H,14,16)"
nvdp = "C1CC(N(C1)C(=O)CNCCNC2=NC=C(C=C2)C#N)C#N"
dimefum = "InChI=1S/C6H8O4/c1-9-5(7)3-4-6(8)10-2/h3-4H,1-2H3/b4-3+"
g12 = "ClC1=CC=CC2=CC=CC(N3CC4=C(CC3)C(N5CC6C(CC5)OC6=O)=NC(OC[C@@H]7CCCN7C)=N4)=C21"
falsep = 'InChI=1S/C22H26FN7O4S/c1-3-4-24-21(31)16-10-18(25-20-9-15(2)26-27-20)12-19(11-16)28-5-7-29(8-6-28)22(32)17-13-30(14-17)35(23,33)34/h1,9-12,17H,4-8,13-14H2,2H3,(H,24,31)(H2,25,26,27)'


model_file = "../saved_models/GCN"
model = tf.keras.models.load_model(model_file)
mol = falsep
make_gradcam_heatmap(mol, model=model)

In [ ]:
model.predict(encode(mol))